<a href="https://colab.research.google.com/github/ShubhamTiwary914/MachineLearning_Notebooks/blob/main/3_DataAugmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=mTVf7BN7S8w&list=PLeo1K3hjS3uu7CxAacxVndI4bE_o3BDtO&index=27

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import PIL
import os
import cv2
import numpy as np

In [ ]:
from tensorflow.python.client import device_lib

def get_hardware_info():
    devices = device_lib.list_local_devices()
    for device in devices:
        print(f"Device name: {device.name}")
        print(f"Device type: {device.device_type}")
        print(f"Memory limit: {device.memory_limit}")
        print('')

get_hardware_info()

Device name: /device:CPU:0
Device type: CPU
Memory limit: 268435456

Device name: /device:GPU:0
Device type: GPU
Memory limit: 14626652160



## Import Dataset

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

In [ ]:
#Set Path
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

### Set dataset and labels to dictionaries

In [ ]:
flowers = {
    "daisy": list(data_dir.glob('daisy/*')),
    "dandelion": list(data_dir.glob('dandelion/*')),
    "roses": list(data_dir.glob('roses/*')),
    "sunflowers": list(data_dir.glob('sunflowers/*')),
    "tulips": list(data_dir.glob('tulips/*')),
}

labels = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [ ]:
X, y = [], []
for flower_name, flowers_list in flowers.items():
  for flowerImg in flowers_list:
    #Fit image
    img = cv2.imread(str(flowerImg))
    imgResized = cv2.resize(img,(180,180))
    #Push X->Image and y->label
    X.append(imgResized)
    y.append(labels[flower_name])

X = np.array(X)
y = np.array(y)

### Split into Train-Test sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Scale neuron from 0-255(int) to 0-1(float)

In [ ]:
X_train_scaled= X_train/255
X_test_scaled = X_test/255

In [ ]:
print(X_train.shape)
print(y_train.shape)

(2752, 180, 180, 3)
(2752,)


### Set Data Augmentation Preprocessing

In [ ]:
dataAugment_preprocessor = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(180, 180,3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

### Build CNN Model and train it

In [ ]:
classSize = len(labels)
model = keras.Sequential([
    #Preprocessor
    dataAugment_preprocessor,
    #Hidden CNN Layer-1
    keras.layers.Conv2D(filters=10, kernel_size=3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    #Hidden CNN Layer-2
    keras.layers.Conv2D(filters=25, kernel_size=3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    #Hidden CNN Layer-3
    keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    #Dense Layer
    keras.layers.Flatten(),
    keras.layers.Dense(140, activation='relu'),
    keras.layers.Dense(classSize)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=30)

Epoch 1/30
86/86 [==============================] - 4s 17ms/step - loss: 10.4890 - accuracy: 0.3735
Epoch 2/30
86/86 [==============================] - 2s 21ms/step - loss: 1.1719 - accuracy: 0.5363
Epoch 3/30
86/86 [==============================] - 1s 14ms/step - loss: 1.0348 - accuracy: 0.5934
Epoch 4/30
86/86 [==============================] - 1s 15ms/step - loss: 0.9450 - accuracy: 0.6374
Epoch 5/30
86/86 [==============================] - 1s 15ms/step - loss: 0.8988 - accuracy: 0.6602
Epoch 6/30
86/86 [==============================] - 1s 14ms/step - loss: 0.8402 - accuracy: 0.6719
Epoch 7/30
86/86 [==============================] - 1s 14ms/step - loss: 0.7935 - accuracy: 0.6940
Epoch 8/30
86/86 [==============================] - 1s 15ms/step - loss: 0.7651 - accuracy: 0.7100
Epoch 9/30
86/86 [==============================] - 1s 15ms/step - loss: 0.7650 - accuracy: 0.7169
Epoch 10/30
86/86 [==============================] - 1s 15ms/step - loss: 0.6791 - accuracy: 0.7471
Epoch 11

In [ ]:
model.evaluate(X_test, y_test)

29/29 [==============================] - 1s 15ms/step - loss: 1.2197 - accuracy: 0.6808


[1.2196540832519531, 0.6808279156684875]